## IMPLEMENTACIÓ DE L'ALGORISME AES

Albert Garcia Uría 1637521
Mario Amador Hurtado 1637543

3r Enginyeria de Dades

Criptografia i Seguretat

In [81]:
# Funcións auxiliars
def tohex(a):
    return format(a, "02x") # 2 dígits a hexadecimal

def int_miss(m):
    assert len(m) == 16, "El missatge ha de tenir 16 caràcters." # Requisit 16 caràcters = 4x4
    m_int = np.array([ord(l) for l in m],dtype = np.uint8).reshape(4,4) # Convertir a enter i fer reshape
    return m_int

def int2miss(array):
    return "".join([chr(e) for e in array.flatten()]) # Convertir a string

In [82]:
import numpy as np

m = "Ennio Morricone "
m_int = int_miss(m)
k_int = np.array([[0x00, 0x11, 0x22, 0x33], [0x44, 0x55, 0x66, 0x77], [0x88, 0x99, 0xaa, 0xbb], [0xcc, 0xdd, 0xee, 0xff]], dtype=np.uint8)

# MATRIUS EXEMPLE ANIMACIÓ
# m_int = np.array([[0x32, 0x88, 0x31, 0xe0],[0x43, 0x5a, 0x31, 0x37],[0xf6, 0x30, 0x98, 0x07],[0xa8, 0x8d, 0xa2, 0x34]], dtype=np.uint8)
# k_int = np.array([[0x2b, 0x28, 0xab, 0x09],[0x7e, 0xae, 0xf7, 0xcf],[0x15, 0xd2, 0x15, 0x4f],[0x16, 0xa6, 0x88, 0x3c]], dtype=np.uint8)

print("MISSATGE:", m, "\n")
print("MATRIU DEL MISSATGE:\n", np.vectorize(tohex)(m_int), "\n")
print("MATRIU DEL LA CLAU:\n", np.vectorize(tohex)(k_int))

MISSATGE: Ennio Morricone  

MATRIU DEL MISSATGE:
 [['45' '6e' '6e' '69']
 ['6f' '20' '4d' '6f']
 ['72' '72' '69' '63']
 ['6f' '6e' '65' '20']] 

MATRIU DEL LA CLAU:
 [['00' '11' '22' '33']
 ['44' '55' '66' '77']
 ['88' '99' 'aa' 'bb']
 ['cc' 'dd' 'ee' 'ff']]


In [83]:
# Constants

global sbox
global mixcolmat
global pol
global rcon

pol = int("100011011", 2)

sbox = np.array([
        0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
        0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
        0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
        0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
        0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
        0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
        0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
        0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
        0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
        0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
        0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
        0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
        0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
        0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
        0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
        0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16
], dtype=np.uint8).reshape(16, 16)

mixcolmat =np.array([
    [0x02, 0x03, 0x01, 0x01],
    [0x01, 0x02, 0x03, 0x01],
    [0x01, 0x01, 0x02, 0x03],
    [0x03, 0x01, 0x01, 0x02]
], dtype=np.uint8)

rcon = np.array([0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36], dtype=np.uint8)

In [84]:
import numpy as np

def addroundkey(m, k):
    res = np.bitwise_xor(m, k) # XOR entre matrius
    return res

mk = addroundkey(m_int, k_int)
print("RESULTAT DEL PRIMER ADDROUNDKEY:\n", np.vectorize(tohex)(mk))

RESULTAT DEL PRIMER ADDROUNDKEY:
 [['45' '7f' '4c' '5a']
 ['2b' '75' '2b' '18']
 ['fa' 'eb' 'c3' 'd8']
 ['a3' 'b3' '8b' 'df']]


In [85]:
def map_sbox(el):
    d1 = int(el[0], 16) # Convertir a decimal
    d2 = int(el[1], 16) # Convertir a decimal
    return sbox[d1, d2] # Retornar valor de la sbox

def bytesub(mk):
    res = np.zeros((4, 4))
    mk_hex = np.vectorize(tohex)(mk) # Convertir a hexadecimal
    map = np.vectorize(map_sbox)(mk_hex) # Aplicar la sbox
    return map

bs = bytesub(mk)
print("RESULTAT DEL PRIMER SUBBYTES:\n", np.vectorize(tohex)(bs))

RESULTAT DEL PRIMER SUBBYTES:
 [['6e' 'd2' '29' 'be']
 ['f1' '9d' 'f1' 'ad']
 ['2d' 'e9' '2e' '61']
 ['0a' '6d' '3d' '9e']]


In [86]:
def shiftrows(bs):
    res = np.zeros((4, 4))
    for i in range(4):
        res[i] = np.roll(bs[i], -i) # Rotar les files
    return res.astype(int) # Retornar enter

sr = shiftrows(bs)
print("RESULTAT DEL PRIMER SHIFTROWS:\n", np.vectorize(tohex)(sr))

RESULTAT DEL PRIMER SHIFTROWS:
 [['6e' 'd2' '29' 'be']
 ['9d' 'f1' 'ad' 'f1']
 ['2e' '61' '2d' 'e9']
 ['9e' '0a' '6d' '3d']]


In [87]:
def mult1(el):
    return el

print(mult1(7))
print(format(mult1(7), "08b"))

7
00000111


In [88]:
def mult2(el):
    bin = format(el, "08b") # Convertir a binari
    res = bin[1:] + "0" # Rotar

    if bin[0] == "1": # Si el primer bit és 1
        return (int(res, 2) ^ pol) % 256 # XOR amb polinomi

    return int(res, 2) # Retornar enter

print(mult2(7))
print(format(mult2(7), "08b"))

14
00001110


In [89]:
def mult3(el):
    m2 = mult2(el) # Multiplicar per 2
    return m2 ^ el # XOR amb el resultat de la multiplicació per 2

print(mult3(7))
print(format(mult3(7), "08b"))

9
00001001


In [90]:
def mixcolumns(sr):
    res = np.zeros((4,4))
    for i in range(4): # Per a cada columna
        for j in range(4): # Per a cada fila
            sum = 0 # Inicialitzar suma
            for k in range(4): # Per a cada element de la fila
                if mixcolmat[i][k] == 1:
                    sum ^= mult1(sr[k][j]) # Multiplicar per 1
                elif mixcolmat[i][k] == 2:
                    sum ^= mult2(sr[k][j]) # Multiplicar per 2
                elif mixcolmat[i][k] == 3:
                    sum ^= mult3(sr[k][j]) # Multiplicar per 3
            res[i][j] = sum # Guardar suma
    return res.astype(int) # Retornar enter

print("RESULTAT DEL PRIMER MIXCOLUMNS:\n", np.vectorize(tohex)(mixcolumns(sr)))

RESULTAT DEL PRIMER MIXCOLUMNS:
 [['d0' 'dc' 'fe' 'bb']
 ['a3' '82' '72' '5a']
 ['16' 'ff' '69' 'c1']
 ['26' 'e9' '21' 'bb']]


In [91]:
def key_schedule(k, num_iteracions):
    num_iteracions += 1 # Incrementar el nombre d'iteracions ja que la primera és la clau inicial

    if k.shape != (4, 4): # Comprovar mida (4,4)
        raise ValueError("k ha de ser una matriu de 4x4!")
    
    w = np.zeros((num_iteracions, 4, 4), dtype=np.uint8) # Crear matriu de zeros de les 10 claus
    w[0, :, :] = k # Assignar la clau inicial a la primera posició
    
    for n in range(1, num_iteracions):
        temp = w[n-1, :, 3].copy() # Copiar la columna 3
        temp = np.roll(temp, -1) # Rotar
        temp = bytesub(temp) # Aplicar la sbox
        temp[0] ^= rcon[n-1] # XOR amb la rcon
        w[n, :, 0] = np.bitwise_xor(temp, w[n-1, :, 0]) # XOR amb la primera columna de la clau anterior
        for i in range(1, 4):
            w[n, :, i] = np.bitwise_xor(w[n, :, i-1], w[n-1, :, i]) # XOR amb la columna anterior
    return w

In [92]:
print("RESULTAT DEL KEYSCHEDULE:\n", np.vectorize(tohex)(key_schedule(k_int,10)))

RESULTAT DEL KEYSCHEDULE:
 [[['00' '11' '22' '33']
  ['44' '55' '66' '77']
  ['88' '99' 'aa' 'bb']
  ['cc' 'dd' 'ee' 'ff']]

 [['f4' 'e5' 'c7' 'f4']
  ['ae' 'fb' '9d' 'ea']
  ['9e' '07' 'ad' '16']
  ['0f' 'd2' '3c' 'c3']]

 [['71' '94' '53' 'a7']
  ['e9' '12' '8f' '65']
  ['b0' 'b7' '1a' '0c']
  ['b0' '62' '5e' '9d']]

 [['38' 'ac' 'ff' '58']
  ['17' '05' '8a' 'ef']
  ['ee' '59' '43' '4f']
  ['ec' '8e' 'd0' '4d']]

 [['ef' '43' 'bc' 'e4']
  ['93' '96' '1c' 'f3']
  ['0d' '54' '17' '58']
  ['86' '08' 'd8' '95']]

 [['f2' 'b1' '0d' 'e9']
  ['f9' '6f' '73' '80']
  ['27' '73' '64' '3c']
  ['ef' 'e7' '3f' 'aa']]

 [['1f' 'ae' 'a3' '4a']
  ['12' '7d' '0e' '8e']
  ['8b' 'f8' '9c' 'a0']
  ['f1' '16' '29' '83']]

 [['46' 'e8' '4b' '01']
  ['f2' '8f' '81' '0f']
  ['67' '9f' '03' 'a3']
  ['27' '31' '18' '9b']]

 [['b0' '58' '13' '12']
  ['f8' '77' 'f6' 'f9']
  ['73' 'ec' 'ef' '4c']
  ['5b' '6a' '72' 'e9']]

 [['32' '6a' '79' '6b']
  ['d1' 'a6' '50' 'a9']
  ['6d' '81' '6e' '22']
  ['92' 'f8' '8a' '

In [93]:
def aes(m, k):
    ks = key_schedule(k, 10) # Generar les claus
    for i in range(9): 
        k = ks[i, :, :] # Assignar la clau
        mk = addroundkey(m, k)
        bs = bytesub(mk) 
        sr = shiftrows(bs)
        m = mixcolumns(sr)

    k = ks[i + 1, :, :] # Assignar la clau
    mk = addroundkey(m, k)
    bs = bytesub(mk)
    sr = shiftrows(bs)

    # Omitim mixcolumns a la última iteració

    k = ks[i + 2, :, :] # Assignar la clau
    res = addroundkey(sr, k)

    return res

res = np.vectorize(tohex)(aes(m_int, k_int))
print("RESULTAT FINAL DE L'ALGORISME:\n", res)

RESULTAT FINAL DE L'ALGORISME:
 [['e0' '9d' '57' '8a']
 ['9a' '31' '4d' '26']
 ['eb' '2d' 'bf' '33']
 ['c0' '78' 'd9' '4f']]


In [94]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import numpy as np

data = m_int.T.tobytes() # Convertir a bytes
key = k_int.T.tobytes() 

cipher = AES.new(key, AES.MODE_ECB) # Crear objecte AES
ciphertext = cipher.encrypt(data) # Encriptar

encriptat = np.frombuffer(ciphertext, dtype=np.uint8).reshape((4, 4)).T # Convertir a matriu
enc = np.vectorize(tohex)(encriptat) # Convertir a hexadecimal
print("Misstage encriptat:\n")
print(enc)
print("\nComparació del nostre codi amb el de la llibreria Crypto:", np.all(res == enc), "\n")

cipher = AES.new(key, AES.MODE_ECB) # Crear objecte AES
deciphertext = cipher.decrypt(ciphertext) # Desencriptar

desencriptat = np.frombuffer(deciphertext, dtype=np.uint8).reshape((4, 4)).T # Convertir a matriu
desc = np.vectorize(tohex)(desencriptat) # Convertir a hexadecimal
print("Missatge desxifrat:\n")
print(desc)
print("\nMissatge desencriptat:", int2miss(desencriptat))


Misstage encriptat:

[['e0' '9d' '57' '8a']
 ['9a' '31' '4d' '26']
 ['eb' '2d' 'bf' '33']
 ['c0' '78' 'd9' '4f']]

Comparació del nostre codi amb el de la llibreria Crypto: True 

Missatge desxifrat:

[['45' '6e' '6e' '69']
 ['6f' '20' '4d' '6f']
 ['72' '72' '69' '63']
 ['6f' '6e' '65' '20']]

Missatge desencriptat: Ennio Morricone 
